# Szyfrowanie homomorficzne

In [1]:
## Funkcje pomocnicze
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik
    while a != 0:
        a, b = b % a, a
    return b

def lcm(a, b):
    # LCM - Lowest Common Multiple, Najmniejsza wspólna wielokrotność
    return a * b // gcd(a, b)

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m
    # czyli (a*x) % m =1

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny

    # Rozszerzony algorytm Euklidesa
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

### Zadanie 1
Dziełając wyłącznie na szyfrogramach RSA zaproponuj i zaimplementuje metodą fałszującą odszyfrowaną wiadomość tak by była ona liczbą podzielną przez 17.  

In [11]:
def generateRSAKeys(keySize):
    print('1. Generujemy liczby p i q')
    p=49109
    q=40639
    e = 65537
    print('3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)')
    d=findModInverse(e, (p-1)*(q-1))
    n= p*q
    publicKey = (n, e)
    privateKey = (n, d)
    print('Klucz publiczny:', publicKey)
    print('Klucz prywatny:', privateKey)
    return (publicKey, privateKey)

public, private = generateRSAKeys(16)

print('Generujemy klucze publiczny i prywatny:', public, private)

def encryptRSA(data_number, modulus, exp):
    data_encrypted = pow(data_number, exp, modulus)
    return data_encrypted

def decryptRSA(data_number, modulus, exp):
    data_decrypted = pow(data_number, exp, modulus)
    return data_decrypted


cip1 = encryptRSA(26, public[0], public[1])
pla1 = decryptRSA(cip1, private[0], private[1])

cip2 = encryptRSA(17, public[0], public[1])
pla2 = decryptRSA(cip2, private[0], private[1])


cip3 = cip1 * cip2
pla3 = decryptRSA(cip3, private[0], private[1])
print(pla3, 26 * 17)


1. Generujemy liczby p i q
3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)
Klucz publiczny: (1995740651, 65537)
Klucz prywatny: (1995740651, 601158737)
Generujemy klucze publiczny i prywatny: (1995740651, 65537) (1995740651, 601158737)
442 442


## Kryptosystem Pailliera

W 1999 roku Pascal Paillier zaproponował kryptosystem, którego bezpieczeństwo oparte było o problem faktoryzacji liczb całkowitych i problem logarytmu dyskretnego.


#### Generowanie kluczy (wersja podstawowa)

1. Wybierz losowo dwie duże liczby pierwsze $p$ i $q$ tak aby $gcd(pq, (p-1)(q-1)) = 1$.
2. Oblicz $n = pq$
3. Oblicz $\lambda = lcm(p-1,q-1)$ ($lcm$ --- Least Common Multiple, Najmniejsza Wspólna Wielokrotność)
4. Zdefiniuj funkcje $L(x) = \frac{x-1}{n}$
5. Wybierz losowo $g \in Z^*_{n^2}$ (liczba całkowita w zakresie 1 do $n^2$)
6. Oblicz odwrotność multiplikatywną $\mu = L(g^\lambda \bmod n^2))^{-1} \bmod n$. Jeśli $\mu$ nie istnieje zacznij od nowa.

Klucz publiczny ma postać: $pk = (n,g)$
Klucz prywatny ma postać: $sk = (\lambda, \mu)$

#### Generowanie kluczy wersja uproszczona
Jeśli $p$ i $q$ są podobniej długości można użyć prostszego wariantu:
1. $g = n+1$
2. $\lambda = \phi(n)$
3. $\mu = \phi(n)^{-1} \bmod n$

#### Szyfrowanie

1. Tekstem jawnym jest liczba $m$ zakresu $0 \le m < n$.

2. Wybierz losową liczbę z zakresu $0 \le r < n$ oraz względnie pierwszą z $n$

3. Oblicz szyfrogram $c= g^m \cdot r^n \bmod n^2$

#### Deszyfrowanie

1. Szyfrogram musi być liczbą z zakresu $0 < c < n^2$
2. Oblicz tekst jawny $m= L(c^\lambda \bmod n^2)\cdot \mu \bmod n$

### Homomorficzne własności schematu Pailliera

1. Dodawanie dwóch liczb:
$$D_{priv}(E_{pub}(m_1) \cdot E_{pub}(m_2) \bmod n^2)= m_1 + m_2 \bmod n$$

2. Mnożenie szyfrogramu przez liczbę:
$$D_{priv}(E_{pub}(m_1)^{m_2} \bmod n^2)= m_1 \cdot m_2 \bmod n$$

### Zadanie 2
Zaimplementuj szyfrowanie i deszyfrowanie Pailliera oraz funkcje umożliwiające homomorficzne operacja dodawania oraz mnożenia przez liczbę całkowitą. Wykaż poprawność operacji homomorficznych.

In [28]:
def genPaillierKeys():
    p = 49109
    q = 40639
    nt = p*q
    gt = nt+1

    lambdat = (p-1) * (q-1)
    mut = pow(lambdat,-1,nt)
    return nt, gt, lambdat, mut

n, g, Lambda, Mu = genPaillierKeys()

def encryptPaillier(m, n, g):
  return pow(g,m,n*n) * pow(3,n,n*n)

def decryptPaillier(c, Lambda, Mu, n):
  l = (pow(c,Lambda,n*n)-1)//n
  m = (l* Mu) % n
  return m

cip1 = encryptPaillier(13,n,g)
pla1 = decryptPaillier(cip1, Lambda, Mu,n)
print(pla1)

cip2 = encryptPaillier(33,n,g)
pla2 = decryptPaillier(cip2, Lambda, Mu,n)
print(pla2)


cip3 = cip1 * cip2
pla3 = decryptPaillier(cip3, Lambda, Mu,n)
print(pla3, 13 + 33)



13
33
46 46


## Szyfrowanie w Pełni Homomorficzne *Fully Homomorphic Encryption*
Szyfrowanie w Pełni Homomorficzne umożliwia realizację wszystkicj operacji arytmetycznych czyli dodawania i mnożenia.

Wykorzystaj bibliotekę Pyfhel w celu sprawdzenia możliwości realizacji operacji matematycznych pod osłoną szyfrowania homomorficznego.

https://pyfhel.readthedocs.io/en/latest/

In [29]:
!pip install pyfhel
import numpy as np
from Pyfhel import Pyfhel, PyPtxt, PyCtxt

HE = Pyfhel()
HE.contextGen(scheme='bfv', n=2**14, t_bits=20)
HE.keyGen()

int1 = np.array([127])
int2 = np.array([-2])
ciphertext1 = HE.encryptInt(int1)
ciphertext2 = HE.encryptInt(int2)


ciphertextSum = ciphertext1 + ciphertext2
summ = HE.decryptInt(ciphertextSum)
print(f"Suma = {summ}")

Suma = [125   0   0 ...   0   0   0]


In [30]:

HE = Pyfhel()           # Creating empty Pyfhel object
ckks_params = {
    'scheme': 'CKKS',   # can also be 'ckks'
    'n': 2**14,         # Polynomial modulus degree. For CKKS, n/2 values can be
                        #  encoded in a single ciphertext.
                        #  Typ. 2^D for D in [10, 15]
    'scale': 2**30,     # All the encodings will use it for float->fixed point
                        #  conversion: x_fix = round(x_float * scale)
                        #  You can use this as default scale or use a different
                        #  scale on each operation (set in HE.encryptFrac)
    'qi_sizes': [60, 30, 30, 30, 60] # Number of bits of each prime in the chain.
                        # Intermediate values should be  close to log2(scale)
                        # for each operation, to have small rounding errors.
}
HE.contextGen(**ckks_params)  # Generate context for ckks scheme
HE.keyGen()             # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()


float1 = np.array([1.2])
float2 = np.array([2.2])
ciphertext1 = HE.encryptFrac(float1)
ciphertext2 = HE.encryptFrac(float2)

ciphertextSum = ciphertext1 + ciphertext2
summ = HE.decryptFrac(ciphertextSum)
print(f"Suma = {summ}")

Suma = [ 3.40000013e+00  7.92893715e-07  1.31974816e-07 ...  2.16371807e-06
  1.25236545e-07 -9.17861997e-08]


## Zadanie 3
Napisz program, który sumuje zawartość tablicy wypełnionej losowymi wartościa całkowitymi z przedziału od 0 do 100. Sumowanie ma być wykonane pod ochroną szyfrowania homomorficznego.


In [73]:
import random, time

random_array = [random.randint(0,100) for _ in range(6)]
print(f"Random array: {random_array}")

start_time = time.time()

HE = Pyfhel()
HE.contextGen(scheme='bfv', n=2**14, t_bits=20)
HE.keyGen()


ciphertextSum = 0
for _,c in enumerate(random_array):
    int1 = np.array([c])
    ciphertextSum += HE.encryptInt(int1)

summ = HE.decryptInt(ciphertextSum)


end_time = time.time()
print(f"Calculation time: {end_time - start_time} ms")
print(f"Found sum {summ[0]}")


Random array: [24, 45, 58, 74, 20, 19]
Calculation time: 0.5229921340942383 ms
Found sum 240


## Zadanie 4
Oceń wydajność operacji wykonywanych na zaszyfrowanych danych i porównaj z operacjami wykonywamy na wiadomościach jawnych.

In [74]:
start_time = time.time()

summ = 0
for _,c in enumerate(random_array):
    summ += c

end_time = time.time()
print(f"Calculation time: {end_time - start_time} ms")
print(f"Found sum {summ}")

Calculation time: 0.00021886825561523438 ms
Found sum 240


## Zadanie 5
Napisz program, który oblicza średnią arytmetyczną losowych wartości rzeczywistych z przedziału od 0,0 do 100.0. Obliczenie ma być wykonane pod ochroną szyfrowania homomorficznego.